M43  - Display pin status, watch pins for changes, watch endstops & toggle LED, Z servo probe test, toggle pins

In [1]:
%pylab inline
import re
import serial
from time import sleep
from mypy.types import Union
from collections import namedtuple

Populating the interactive namespace from numpy and matplotlib


In [2]:
ctlport = '/dev/ttyUSB2'
ctl = serial.Serial(ctlport, baudrate=250000, timeout=0)
sensorport = '/dev/ttyACM0'
sensor = serial.Serial(sensorport, baudrate=115200, timeout=0)

In [3]:
print(ctl.readline())
while ctl.in_waiting:
    print(ctl.readline())

b''


In [4]:
ctl.write(b'M114\r\n')
sleep(0.25)
ctl.read(1000)

b''

In [5]:
def cmd_async(command):
    if type(command) != bytes:
        command = command.encode()
    ctl.write(command)
    ctl.write(b'\r\n')
    ctl.flush()
    
def fold_lines(fragments):
    buffer = b''
    for frag in fragments:
        buffer += frag
        if buffer.endswith(b'\n'):
            yield buffer
            buffer = b''
    if buffer:
        yield buffer

def cmd(command, do_print=False):
    if type(command) != bytes:
        command = command.encode()
    ctl.reset_input_buffer()
    ctl.write(command)
    ctl.write(b'\r\n')
    ctl.flush()
    rv = []
    while True:
        line = ctl.readline()
        if line:
            rv.append(line)
            if line.strip().lower() == b'ok':
                break
    if do_print:
        print('\n'.join(line.decode().strip() for line in rv))
    return list(fold_lines(rv))

In [ ]:
# Home
cmd('G28')
# Set units to mm
cmd('G21')
# Absolute positioning
cmd('G90')

KeyboardInterrupt: 

In [ ]:
def go(x=None,y=None,z=None,speed=None):
    tmpl = 'G0'
    if x is not None:
        tmpl += ' X{x}'
    if y is not None:
        tmpl += ' Y{y}'
    if z is not None:
        tmpl += ' Z{z}'
    if speed is not None:
        tmpl += ' F{speed}'
    s = tmpl.format(x=x, y=y, z=z, speed=speed)
    cmd(s)
    return cmd('G4 P0')

In [ ]:
def check_hit():
    last_sign = None
    try:
        while True:
            line = sensor.readline().strip()
            if not line: 
                break
            ambient, u, v = [float(_) for _ in line.split(b';')]
            u -= ambient
            v -= ambient
            current_sign = sign(u-v)
            if last_sign is not None and \
               (current_sign != last_sign):
                    return True
            last_sign = current_sign
        return False
    except:
        return False

In [ ]:
Position = namedtuple('Position',['x','y','z'])
def current_position():
    raw = cmd('M114')[0]
    m = re.search(b'^X:(.*?) Y:(.*?) Z:(.*?) E:', raw)
    x,y,z = [float(_) for _ in m.groups()]
    return Position(x,y,z)
current_position()

In [ ]:
def probe_coarse(z1, z2):
    # Go piecewise from z1 to z2, check endstop for each segment
    # Return position at which endstop was hit approximately
    # There is no G-code to stop precisely immediately, so go in segments
    z1, z2 = sorted([z1,z2])
    span = abs(z2-z1)
    segment_length = 1.0 # mm
    num_segments = int(ceil(span / segment_length))
    zz = linspace(z1,z2,num_segments)[1:]
    go(z=z1)
    for z in zz:
        cmd_async('G0 Z{z} F100'.format(z=z))
        while True:
            if check_hit():
                rv = cmd('M114')
                # Wait until move done
                cmd('M400')
                return rv
            else:
                # Did we reach target z?
                if np.isclose(current_position().z, z, atol=0.01):
                    break
    return None

go(z=10,speed=9999)
go(x=100,y=100)
print(probe_coarse(10, 1))

In [ ]:
go(z=3)
sensor.reset_input_buffer()
#cmd_async('M0 Z2')
for i in range(20):
    z = current_position().z
    print(z, check_hit(), flush=True)
    if z<2.05: 
        break
    sleep(0.05)

In [ ]:
ctl.write(b'M114\n')
ctl.read(1000)

In [ ]:
EndstopNotFound = object()
EndstopResult = Union[EndstopNotFound, float]

def probe_subdivide(z1:float, z2:float, epsilon:float=0.01, was_hit:bool=False) -> EndstopResult :
    zmin, zmax = sorted([z1,z2])
    # Delta-z: Half subdivision span
    dz = (zmax-zmin)/2
    if(dz<epsilon and was_hit):
        return (z2+z1)/2
    if move_check_hit(zmax, zmin+dz):
        return find_endstop(zmax, zmin+dz, epsilon, was_hit=True)
    elif move_check_hit(zmin+dz, zmin):
        return find_endstop(zmin+dz, zmin, epsilon, was_hit=True)
    else:
        return EndstopNotFound
    
probe_subdivide(50,2)

In [ ]:
x,y = 100, 100
zmin = 1
zmax = 50
go(z=zmax)
go(x=100, y=100)
probe()

In [ ]:
#sensor.reset_input_buffer()
sensor.readline()

In [ ]:
sign([-1,0,1])